# How to Calculate a Game's Box Score using Play-by-Play Data

### Import Packages

In [1]:
import re
import datetime
import numpy as np
import pandas as pd
from functools import reduce
pd.set_option('precision', 2)

### Import Data

In [2]:
hou_gsw = '[2017-10-17]-0021700002-HOU@GSW.csv'
bos_cle = '[2017-10-17]-0021700001-BOS@CLE.csv'

df = pd.read_csv('../data/2017-18_pbp/{}'.format(bos_cle))
df.head(3)

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
0,"=""0021700001""",2017-2018 Regular Season,2017-10-17,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,NaN,NaN,start of period,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""0021700001""",2017-2018 Regular Season,2017-10-17,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Love vs. Horford: Tip to Irving
2,"=""0021700001""",2017-2018 Regular Season,2017-10-17,Gordon Hayward,Jayson Tatum,Al Horford,Jaylen Brown,Kyrie Irving,LeBron James,Jae Crowder,...,NaN,made,NaN,unknown,10.0,-1.0,100.0,25.1,15.0,Irving 10' Driving Floating Jump Shot (2 PTS) ...


In [3]:
df.shape

(477, 44)

### Data Cleaning

##### Fill the Null Values

In [4]:
# Only filling the null values in Points for now
df.points = df.points.fillna(0)

##### Clean the Event Type Column

In [5]:
# Remove the 'event_type' : 'end of period', 'start of period'
bad_events = ['start of period', 'end of period']
df = df[~df.event_type.isin(bad_events)]

##### Reformat the Time Columns

In [6]:
# Change play length to seconds (float)
df['play_length'] = pd.to_timedelta(df['play_length'])
df['play_length'] = df['play_length'].dt.total_seconds()

# Change the other time columns to time deltas
df['elapsed'] = pd.to_timedelta(df['elapsed'])
df['remaining_time'] = pd.to_timedelta(df['remaining_time'])

##### Reformat the Player Columns

In [7]:
# Create dummy variables for the player columns
player_df = pd.get_dummies(df.filter(regex='a[1-5]|h[1-5]'), prefix='player')

# Remove the whitespace in the dummy player columns
player_df.columns = [x.strip().replace(' ', '_') for x in player_df.columns]

# Collapse the duplicate dummy player columns and sum the column values
player_df = player_df.groupby(lambda x:x, axis=1).sum()

# Bring the dummy columns into the main dataframe
df = pd.concat([df, player_df], axis=1)

### Calculate Box Score Stats

##### Field Goals

In [8]:
fg_df = df[~df.event_type.isin(['free throw'])]

fg_df = pd.pivot_table(fg_df, index=['player'], 
                              columns=['result'],
                              values=['play_id'], 
                              aggfunc='count').reset_index(col_fill='player')

fg_df.columns = fg_df.columns.droplevel(0)
fg_df.columns.name = None
fg_df['missed'] = fg_df['missed'].fillna(0)
fg_df['fga'] = fg_df['made'] + fg_df['missed']
fg_df = fg_df.rename(columns={'made':'fg'}).drop('missed', axis=1)
fg_df['fg%'] = fg_df['fg'] / fg_df['fga']

##### Free Throws

In [9]:
ft_df = df[df.event_type.isin(['free throw'])]

ft_df = pd.pivot_table(ft_df, index=['player'], 
                              columns=['result'],
                              values=['play_id'], 
                              aggfunc='count').reset_index(col_fill='player')

# Data Cleaning
ft_df.columns = ft_df.columns.droplevel(0)
ft_df.columns.name = None
ft_df['made'] = ft_df['made'].fillna(0)
ft_df['missed'] = ft_df['missed'].fillna(0)
ft_df['fta'] = ft_df['made'] + ft_df['missed']
ft_df = ft_df.rename(columns={'made':'ft'}).drop('missed', axis=1)
ft_df['ft%'] = ft_df['ft'] / ft_df['fta']

##### 3-Pointers

In [10]:
three_df = df[df.description.str.contains('3PT').fillna(False)]

three_df = pd.pivot_table(three_df, index=['player'],
                          columns=['result'],
                          values=['play_id'],
                          aggfunc='count').reset_index(col_fill='player')

three_df.columns = three_df.columns.droplevel(0)
three_df.columns.name = None
three_df['made'] = three_df['made'].fillna(0)
three_df['3pa'] = three_df['made'] + three_df['missed']
three_df = three_df.rename(columns={'made':'3p'}).drop('missed', axis=1)
three_df['3p%'] = three_df['3p'] / three_df['3pa']

##### Rebounds

In [11]:
reb_df = df[df.type.isin(['rebound offensive', 'rebound defensive'])]

reb_df = pd.pivot_table(reb_df, index=['player'],
                                columns=['type'],
                                values=['play_id'],
                                aggfunc='count').reset_index(col_fill='player')

reb_df.columns = reb_df.columns.droplevel(0)
reb_df.columns.name = None
reb_df = reb_df.rename(columns={'rebound defensive':'drb',
                                'rebound offensive':'orb'})
reb_df['drb'] = reb_df['drb'].fillna(0)
reb_df['orb'] = reb_df['orb'].fillna(0)
reb_df['trb'] = reb_df['drb'] + reb_df['orb']

##### Assists

In [12]:
ast_df = pd.DataFrame(df.assist.value_counts()).reset_index().rename(columns={'index':'player',
                                                                              'assist':'ast'})

##### Steals

In [13]:
stl_df = pd.DataFrame(df.steal.value_counts()).reset_index().rename(columns={'index':'player',
                                                                             'steal':'stl'})

##### Blocks

In [14]:
blk_df = pd.DataFrame(df.block.value_counts()).reset_index().rename(columns={'index':'player',
                                                                              'block':'blk'})

##### Turnovers

In [15]:
to_df = df[df.event_type == 'turnover']
to_df = pd.DataFrame(to_df.groupby('player')['event_type'].count()).reset_index().rename(columns={'event_type':'tov'})

##### Fouls

In [16]:
# types = sorted(df.type.unique())
# foul_types = [s for s in types if "foul" in s]
# print(foul_types)

In [17]:
fouls = ['foul', 'off.foul', 'offensive charge foul', 'p.foul', 's.foul']
pf_df = df[df.type.isin(fouls)]
pf_df = pd.pivot_table(pf_df, index=['player'], values=['play_id'], aggfunc='count').reset_index().rename(columns={'play_id':'pf'})

##### Points

In [18]:
pts_df = pd.pivot_table(df, index=['team', 'player'], values=['points'], aggfunc='sum').reset_index()
pts_df = pts_df.rename(columns={'points':'pts'})

##### Minutes Played

In [19]:
# Initialize minutes played dataframe
min_df = pd.DataFrame(columns=['player'])

# Get list of players
player_list = [i for i in list(df.columns) if re.search(r'player_.+_.+', i)]

# Iterate through player list and groupby play length
for player in player_list:
    grouper = df.groupby(player)['play_length'].sum() 
    min_df = min_df.append(grouper, ignore_index=True)

# Reformat the player column
min_df['player_unformatted'] = player_list #[i.replace('player_','').replace('_',' ') for i in player_list]
min_df['player'] = [i.replace('player_','').replace('_',' ') for i in player_list]

# Reformat the minutes played, minutes on bench columns
min_df = min_df.rename(columns={0:'min_bench', 1:'min_played'})

# Reformat the time floats to minutes, seconds
mp_list = []

for time in list(min_df['min_played']):
    output = str(datetime.timedelta(seconds=time))
    mp_list.append(output[2:])

min_df['min_played'] = mp_list

##### Plus/Minus

For a given player: count up points scored by the player’s team and points scored against the player’s team when that player is on the floor. Then simply subtract points against from points for. 

In [20]:
pm_df = pd.merge(pts_df, min_df, on='player')
pm_df.head()

p_list = list(pm_df.player_unformatted)
t_list = list(pm_df.team)

plus = []
minus = []

for player, team in zip(p_list, t_list):
    p_nested = []
    plus.append(p_nested)
    
    m_nested = []
    minus.append(m_nested)
    
    for i, row in df.iterrows():
        if (row[player] == 1) & (row['team'] == team):
            p_nested.append(row['points'])
        elif (row[player] == 1) & (row['team'] != team):
            m_nested.append(row['points'])

p_list = []
m_list = []


for value in plus:
    value = np.sum(value)
    p_list.append(value)
    
for value in minus:
    value = np.sum(value)
    m_list.append(value)
    
pm_df['plus'] = p_list
pm_df['minus'] = m_list
pm_df['plus_minus'] = pm_df['plus'] - pm_df['minus']

### Create the Box Score

In [21]:
# Merge all the dataframes
dfs = [pm_df, fg_df, ft_df, three_df, reb_df, ast_df, stl_df, blk_df, to_df, pf_df]
boxscore = reduce(lambda left, right: pd.merge(left, right, on=['player'], how='outer'), dfs).fillna(0)

# Drop unnecessary columns
drop_cols = ['min_bench', 'player_unformatted', 'plus', 'minus']
boxscore.drop(drop_cols, axis=1, inplace=True)

# Reorder columns for visuals
reorder_cols = ['team', 'player', 'min_played', 'fg', 'fga', 'fg%',
                '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb', 'drb', 
                'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus']

boxscore = boxscore[reorder_cols]

# Reformat column names for visuals
boxscore = boxscore.rename(columns={"min_played":"mp", "plus_minus":"+/-"})
boxscore = boxscore.sort_values('team', ascending=True).reset_index(drop=True)

In [22]:
boxscore

,team,player,mp,fg,fga,fg%,3p,3pa,3p%,ft,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-
0,BOS,Al Horford,32:05,2.0,7.0,0.29,0.0,2.0,0.00,5.0,...,0.0,7.0,7.0,5.0,0.0,1.0,0.0,2.0,9.0,4.0
1,BOS,Terry Rozier,19:32,2.0,6.0,0.33,1.0,3.0,0.33,4.0,...,0.0,3.0,3.0,2.0,4.0,0.0,0.0,0.0,9.0,5.0
2,BOS,Semi Ojeleye,08:39,0.0,0.0,0.00,0.0,1.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-10.0
3,BOS,Marcus Smart,35:03,5.0,16.0,0.31,0.0,4.0,0.00,2.0,...,0.0,9.0,9.0,3.0,2.0,2.0,2.0,2.0,12.0,-9.0
4,BOS,Kyrie Irving,39:21,8.0,17.0,0.47,4.0,9.0,0.44,2.0,...,2.0,2.0,4.0,10.0,3.0,0.0,2.0,5.0,22.0,-1.0
5,BOS,Shane Larkin,04:49,0.0,0.0,0.00,0.0,1.0,0.00,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6,BOS,Jayson Tatum,36:32,5.0,12.0,0.42,1.0,2.0,0.50,3.0,...,4.0,6.0,10.0,3.0,0.0,0.0,1.0,3.0,14.0,7.0
7,BOS,Jaylen Brown,39:37,11.0,23.0,0.48,2.0,9.0,0.22,1.0,...,1.0,5.0,6.0,0.0,2.0,0.0,3.0,5.0,25.0,-4.0
8,BOS,Gordon Hayward,05:15,1.0,2.0,0.50,0.0,1.0,0.00,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0
9,BOS,Aron Baynes,19:07,2.0,2.0,1.00,0.0,0.0,0.00,2.0,...,2.0,3.0,5.0,1.0,0.0,1.0,2.0,4.0,6.0,-12.0


In [23]:
# TODO: Refix errors in columns
# Plus_Minus

# TODO: Determine starters/reserves